In [1]:
pip install simpy

In [2]:
import simpy

In [27]:
def resource_user(name, env, resource, wait, prio):
  yield env.timeout(wait)
  with resource.request(priority=prio) as req:
    print(f'{name} requesting at {env.now} with priority={prio}')
    yield req
    print(f'{name} got resource at {env.now}')
    try:
      yield env.timeout(3)
    except simpy.Interrupt as interrupt:
      by=interrupt.cause.by
      usage=env.now-interrupt.cause.usage_since
      print(f'{name} got preempted {by} at {env.now}'
      f' after {usage}')

In [26]:
env=simpy.Environment()
res=simpy.PreemptiveResource(env,capacity=1)
p1=env.process(resource_user(1, env, res, wait=0, prio=0))
p2=env.process(resource_user(2, env, res, wait=2, prio=-1))
p3=env.process(resource_user(3, env, res, wait=0, prio=2))
p4=env.process(resource_user(4, env, res, wait=1, prio=1))
p5=env.process(resource_user(5, env, res, wait=1, prio=-3))
p6=env.process(resource_user(6, env, res, wait=0, prio=2))
env.run()

1 requesting at 0 with priority=0
3 requesting at 0 with priority=2
6 requesting at 0 with priority=2
1 got resource at 0
4 requesting at 1 with priority=1
5 requesting at 1 with priority=-3
1 got preempted <Process(resource_user) object at 0x7fa7492e7390> at 1 after 1
5 got resource at 1
2 requesting at 2 with priority=-1
2 got resource at 4
4 got resource at 7
3 got resource at 10
6 got resource at 13
